In [1]:
#import dependencies
import os
from PIL import Image
import requests
import matplotlib.pyplot as plt
import torch
import datasets.transforms as T
torch.set_grad_enabled(False);
from models import build_model

In [2]:
# COCO classes
CLASSES = [
    'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
    'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
    'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
    'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
]

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

In [3]:
# Sof-DETR transforms for input image
sof_transform = T.Compose([
    T.RandomResize([800], max_size=1333),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [4]:

# bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

In [5]:
# for plotting and saving it
def plot_results(pil_img, prob, boxes,name,save=False):
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    if save == True:

        plt.savefig(name+"_o.png",bbox_inches='tight',dpi=600)
    ax = plt.gca()
    colors = COLORS * 100
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        cl = p.argmax()
        text = f'{CLASSES[cl]}: {p[cl]:0.2f}'
        ax.text(xmin, ymin, text, fontsize=10,
                bbox=dict(facecolor='red', alpha=0.5))
    plt.axis('off')
    if save==True:

        plt.savefig((name+"_im.png"),bbox_inches='tight',dpi=600)
    plt.show()
    

In [6]:
# Sof-DETR Model-building
class Args():
    def __init__(self):
        self.ArgumentParser='Sof-DETR'
        self.lr=1e-4
        self.lr_backbone=1e-5
        self.batch_size=3 
        self.weight_decay=1e-4
        self.epochs=500
        self.lr_drop=300
        self.clip_max_norm=0.1

        # Model parameters
        self.frozen_weights=''
        # * Backbone
        self.backbone='resnet50'
        self.dilation=False
        self.position_embedding='sine'

        # * Transformer
        self.enc_layers=6
        self.dec_layers=6
        self.dim_feedforward=2048
        self.hidden_dim=256
        self.dropout=0.1
        self.nheads=8
        self.num_queries=100
        self.pre_norm=False

        # * Segmentation
        self.masks=False
        self.return_intermediate=True
        # Loss
        self.aux_loss=False
        # * Matcher
        self.set_cost_class=1
        self.set_cost_bbox=5
        self.set_cost_giou=2
        # * Loss coefficients
        self.mask_loss_coef=1
        self.dice_loss_coef=1
        self.bbox_loss_coef=5
        self.giou_loss_coef=2
        self.eos_coef=0.1
        # dataset parameters
        self.dataset_file='coco'
        self.coco_path=''
        self.coco_panoptic_path=''
        self.remove_difficult=False

        self.output_dir=''
        self.device='cuda'
        self.seed=42
        self.resume=''
        self.start_epoch=0
        self.eval=False
        self.num_workers=2

        # distributed training parameters
        self.world_size=1
        self.dist_url='env://'
args=Args()

model, criterion, postprocessors = build_model(args)
# model.to(args.device)
sof_model = model


In [7]:
#load pre-trained Sof-DETR Model
sof_model=torch.load('../trained/checkpoint0499.pth')
sof_detr_model.load_state_dict(sof_model['model'])
sof_detr_model.eval();

NameError: name 'sof_detr_model' is not defined

In [ ]:
def tuple_of_tensors_to_tensor(tuple_of_tensors):
    return  torch.stack(list(tuple_of_tensors), dim=0)

In [ ]:
# path to your image

# url = 'http://images.cocodataset.org/val2017/000000150224.jpg' # 1
url = 'http://images.cocodataset.org/val2017/000000332351.jpg'#2
im = Image.open(requests.get(url, stream=True).raw)
img_path,image_name=os.path.split(url)
save_name=image_name[:-4]

In [ ]:
# transform the inpit image and nomlaize it 
img_temp = sof_transform(im,None)
img=tuple_of_tensors_to_tensor(img_temp[0]).unsqueeze(0)
# propagate through the model
detections = sof_detr_model(img)

# keeping only predictions with 0.9+ confidence value
prob = detections['pred_logits'].softmax(-1)[0, :, :-1]
keep = prob.max(-1).values > 0.9

# scaling back the bounding boxes 
bboxes_scaled = rescale_bboxes(detections['pred_boxes'][0, keep], im.size)
plot_results(im, prob[keep], bboxes_scaled,save_name,save=False)

In [ ]:
# path to your image

url = 'http://images.cocodataset.org/val2017/000000150224.jpg' # 1
# url = 'http://images.cocodataset.org/val2017/000000332351.jpg'#2
im = Image.open(requests.get(url, stream=True).raw)
img_path,image_name=os.path.split(url)
save_name=image_name[:-4]

In [ ]:
# transform the inpit image and nomlaize it 
img_temp = sof_transform(im,None)
img=tuple_of_tensors_to_tensor(img_temp[0]).unsqueeze(0)
# propagate through the model
detections = sof_detr_model(img)

# keeping only predictions with 0.9+ confidence value
prob = detections['pred_logits'].softmax(-1)[0, :, :-1]
keep = prob.max(-1).values > 0.9

# scaling back the bounding boxes 
bboxes_scaled = rescale_bboxes(detections['pred_boxes'][0, keep], im.size)
plot_results(im, prob[keep], bboxes_scaled,save_name,save=False)